Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [3]:
from random import choices, randint

import lab9_lib

In [2]:
fitness = lab9_lib.make_problem(10)                                    #fitness is an instance of the class Problem (subclass of AbstractProblem)
for n in range(10):
    ind = choices([0, 1], k=50)                                        #ind is the genome
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")       #fitness(ind) calls the function __call__ which calculates the fitness value

print(fitness.calls)

10001110111101001100001100111011010001010110101000: 7.33%
11111111110111101101011001001011111101100010011011: 9.11%
10011011011110010000000011101100011010111111100001: 15.33%
11100001101001001011110111100101011100000010010110: 29.56%
00100100001000011001000010101111101111000000101000: 7.34%
11111100000010010111011011101111001010101101010001: 15.33%
00110110110110011110000000001100001011110011010000: 7.33%
01000000111010101001101000000000011000111111000001: 7.34%
00011101110100101100100101100110111000111011011111: 15.33%
10110101000001001000001110011001101011010110101010: 7.33%
10


In [4]:
def mutate(individual):
    mutation_point = randint(0, len(individual) - 1)
    individual[mutation_point] = 1 - individual[mutation_point]  # Flip 0 to 1 and vice versa
    return individual

In [5]:
def one_plus_one_es(problem, iterations):
    genome_length = 1000
    best_individual = choices([0, 1], k=genome_length)
    best_fitness = problem(best_individual)
    
    for _ in range(iterations):
        mutated_individual = mutate(best_individual.copy())
        mutated_fitness = problem(mutated_individual)

        if mutated_fitness >= best_fitness:
            best_individual = mutated_individual
            best_fitness = mutated_fitness

    return best_individual, best_fitness

In [7]:
problem_instance = lab9_lib.make_problem(1)  # Replace 1 with 2, 5, or 10 
iterations = 1000  # Adjust the number of iterations as needed

best_solution, best_fitness = one_plus_one_es(problem_instance, iterations)

print(f"Best solution: {''.join(map(str, best_solution))}")
print(f"Best fitness: {best_fitness:.2%}")
print(f"Total fitness calls: {problem_instance.calls}")

Best solution: 1111011011111110011101111010111111110110101111111111011011101111011111111111111111010111111111101101101110111111101011101111111110101111111010011100110111111111110111101110111100111101011011111111111111111111100001111111111101010111111111111100111101101011111111111011111111111111111111001011101111111011111111111111111011111111111111111111010111111111111101111101011111110001111111111011011011000111111110111010111010101101101111101111111111101101100101111101111111101101111111110110111101111110111111100101110011111011111010101110111111111111110111110101111011111101101110101011101101111111101001111111111111111111111000111111101111111111001111111011011101111111101101111111011011110011111111111111110101111011111110111101111111011111111111101111111101011101111011111111111111111111111101111101111110111001111111011111111111101111110111111111111001001110110111111111111010111111011101111111100100010011101111011111101111011110111111100110111111110101111011111101111111111010011111001